In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astropy
import vplanet

ModuleNotFoundError: ignored

In [4]:
!pip install -q condacolab

In [2]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
import sys
print(sys.executable)

/usr/bin/python3.real
